In [1]:
#!/usr/bin/env python

'''
DESCRIPTION
-----------
    Preprocessing raw dataset -> applying scaling, filtering genes.
    
RETURN
------
    {DATASET}_{SCALE}.pck : pck file
        Preprocessed file

EXPORTED FILE(s) LOCATION
-------------------------
    ./data/processed/{EXPERIMENT}/{DATASET}_{SCALE}.pck
'''

# importing default libraries
# import os, argparse, sys
# sys.path.append('./')
import os, sys
ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
os.chdir(ROOT_DIR)
sys.path.append(ROOT_DIR)
# importing scripts in scripts folder
from scripts import settings as ssrp, dataset_scripts as dsrp, path_scripts as psrp, model_scripts as msrp
# importing default libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import *
import pyreadr # imported to read .rds files

scripts/settings.py - PATHS IMPORTED!!!


In [69]:
target_column_index  = -1
dataset              = 'Immune.pck'
bio_knowledge        = 'pbk_layer_hsa.txt'
experiment           = 'exper_pbmc'
scale                = StandardScaler()# FunctionTransformer(np.log1p) #etc, MinMaxScaler(), StandardScaler()

In [70]:
# Importing all prior biological knowledge and combine all genes to create a common gene list
list_gene = None
if (bio_knowledge!=None):
    df_bio = pd.DataFrame(pd.read_csv(os.path.join(ssrp.DIR_DATA_PROCESSED, bio_knowledge)))
    list_gene = list(df_bio['symbol'])

# the output location
loc_output = os.path.join(ssrp.DIR_DATA_PROCESSED, experiment)
psrp.define_folder(loc_=loc_output)

# getting scaling information to include into name of the file
sc_text = str(type(scale)).split("'")[1].split(".")[-1]
for k, i in scale.get_params().items():
    if k=='func':
        sc_text = scale.func.__name__

file_name_output = '.'.join(dataset.split('.')[:-1])+'_'+sc_text+'.pck'
print(file_name_output)
print('FILE FORMAT, ', dataset.split('.')[1])
if dataset.split('.')[1]=='rds':
    df_raw = pyreadr.read_r(os.path.join(ssrp.DIR_DATA_EXTERNAL, experiment, dataset))[None]
elif dataset.split('.')[1]=='pck':
    df_raw = pd.read_pickle(os.path.join(ssrp.DIR_DATA_EXTERNAL, experiment, dataset))
    df_raw = pd.concat([(df_raw.iloc[:, :-1]).astype(float) ,df_raw.iloc[:, -1]], axis=1)
else:
    df_raw = pd.read_csv(os.path.join(ssrp.DIR_DATA_EXTERNAL, experiment, dataset))

Immune_StandardScaler.pck
FILE FORMAT,  pck


In [6]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3412 entries, 0 to 3411
Columns: 17995 entries, A1BG to label
dtypes: float64(17994), object(1)
memory usage: 468.4+ MB


In [72]:
df_raw['a1bg','a1cf','a2m','a2ml1']

KeyError: ('a1bg', 'a1cf', 'a2m', 'a2ml1')

In [8]:


# Importing experiment dataset
df = dsrp.ExperimentDataSet(experiment_dataset=df_raw
                            , target_col_index=target_column_index
                            , filter_gene=list_gene
                            , scaler=scale).run()

print('prior bio shape, ', df_bio.shape)
# df_bio.head()

# Exporting preprocessed dataset
df.to_pickle(os.path.join(loc_output,  file_name_output ))
print('Experiment datasets are exported into ', os.path.join(loc_output, file_name_output))


********* RAW DATASET *********
experiment dataset shape     , (17293, 9438)

len 2988
   -> Common genes are filtered! 2988
   -> StandardScaler() operation implemented!

****** PROCESSED DATASET ******
experiment dataset shape     , (17293, 1649)

prior bio shape,  (2987, 94)
Experiment datasets are exported into  ./data/processed/exper_mouse/mouse_retrieval_StandardScaler.pck
